In [1]:
!pip install pandas streamlit firebase-admin


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [2]:
from google.colab import files
import os

In [ ]:
import io
# Subir el archivo CSV
uploaded = files.upload()

# Obtener el nombre del archivo cargado
csv_filename = next(iter(uploaded))

# Leer el archivo CSV en un DataFrame de Pandas
df = pd.read_csv(io.StringIO(uploaded[csv_filename].decode('utf-8')))

Saving movies.csv to movies.csv


In [ ]:
# Migración a Firestore
for index, row in df.iterrows():
    db.collection("peliculas").add(row.to_dict())

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
#instala ngrok,
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2024-02-11 15:35:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  35.0MB/s    in 0.4s    

2024-02-11 15:35:31 (35.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [16]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [44]:
#2GCH3J42GOPVRH7AXKRDK2I4XHKRFNKL

get_ipython().system_raw('./ngrok authtoken 2bWJ4dOMWjYeDCUuW4y4TtowR9f_4vQqhZdsCtw4Cr4fKH4hS')

In [65]:
#servidor temporal
get_ipython().system_raw('./ngrok http 8501 &')

In [66]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
'import sys, json; print("Execute the next cell and the go to the following URL: "+json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://9ee7-34-27-145-115.ngrok-free.app


In [68]:
!streamlit run /content/primero.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.27.145.115:8501

  Stopping...
  Stopping...
Exception ignored in: <module 'threading' from '/usr/lib/python3.10/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1567, in _shutdown
    lock.acquire()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/bootstrap.py", line 69, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/server.py", line 399, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 311, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 798, in call_soon_threadsafe
    self._check_closed()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 515, in _check_closed
    raise RuntimeError('Eve

In [67]:
%%writefile primero.py
import streamlit as st

import json
from firebase_admin import initialize_app
import pandas as pd
import streamlit as st
import firebase_admin
from firebase_admin import credentials, firestore


# Ruta completa al archivo JSON de credenciales
ruta_a_tu_credencial = '/content/drive/MyDrive/Colab Notebooks/reto-netflix-a4e46-firebase-adminsdk-l5d20-32210b41c7.json'

# Cargar las credenciales desde el archivo JSON
with open(ruta_a_tu_credencial) as json_file:
    credenciales_json = json.load(json_file)



##

@st.cache_resource(hash_funcs={credentials.Certificate: id})
def initialize_firestore():
    try:
        initialize_app(credentials.Certificate(credenciales_json))
    except ValueError:
        # La aplicación ya está inicializada, maneja el error según sea necesario
        pass

    # No necesitamos pasar las credenciales al cliente de Firestore
    # Inicializar el cliente de Firestore
    return firestore.client()

# Obtener el cliente de Firestore
db = initialize_firestore()

##
##

# Checkbox para visualizar todos los filmes recuperados
if st.sidebar.checkbox("Mostrar Todos los Filmes"):
    # Obtener todos los filmes de la base de datos
    todos_los_filmes = db.collection(u'peliculas').get()

    # Crear una lista para almacenar los datos de todos los filmes
    data = []

    # Recorrer todos los filmes y obtener sus datos
    for filme in todos_los_filmes:
        filme_data = filme.to_dict()
        data.append(filme_data)

    # Crear un DataFrame a partir de los datos obtenidos
    df_todos_los_filmes = pd.DataFrame(data)

    # Mostrar todos los filmes en forma de tabla
    st.header("Todos los filmes:")
    st.write(df_todos_los_filmes)

##

# Campo de texto para ingresar el título de la película a buscar
titulo_busqueda = st.sidebar.text_input("Buscar por Título")

# Botón de comando para realizar la búsqueda
if st.sidebar.button("Buscar", key="buscar_button"):
    # Convertir el título de búsqueda a minúsculas
    titulo_busqueda = titulo_busqueda.lower()

    # Filtrar los filmes que contengan el título de búsqueda (sin importar mayúsculas o minúsculas)
    filmes_coincidentes = [filme.to_dict() for filme in db.collection(u'peliculas').get() if titulo_busqueda in filme.to_dict()['name'].lower()]



    # Mostrar los datos en una tabla si se encontraron filmes coincidentes
    if filmes_coincidentes:
        # Mostrar el total de registros encontrados
        st.write(f"Total de filmes mostrados: {len(filmes_coincidentes)}")
        df_filmes_coincidentes = pd.DataFrame(filmes_coincidentes)
        st.write(df_filmes_coincidentes)
    else:
        st.write("No se encontraron filmes con ese título.")



##

# Obtener todos los filmes de la base de datos
todos_los_filmes = db.collection(u'peliculas').get()

    # Crear una lista para almacenar los datos de todos los filmes
data = []

    # Recorrer todos los filmes y obtener sus datos
for filme in todos_los_filmes:
    filme_data = filme.to_dict()
    data.append(filme_data)

    # Crear un DataFrame a partir de los datos obtenidos
df_todos_los_filmes_ = pd.DataFrame(data)

# 1. Crear el componente selectbox en el sidebar para cargar la columna 'director'
directores = df_todos_los_filmes_['director'].unique()
selected_director = st.sidebar.selectbox("Seleccionar Director", directores)

# 2. Crear el botón de comando para iniciar la búsqueda
if st.sidebar.button("Buscar por Director", key="buscar_button_"):
    # 3. Definir una función que filtre los filmes realizados por el director seleccionado
    filmes_director = df_todos_los_filmes_[df_todos_los_filmes_['director'] == selected_director]

    # 4. Mostrar el total de filmes encontrados y la tabla con los resultados
    st.write(f'Total de filmes encontrados para el director "{selected_director}": {len(filmes_director)}')
    st.write(filmes_director)


##




Overwriting primero.py
